# 0. 필요 라이브러리 로드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.17.1
1.26.4
4.47.1
3.2.0


In [4]:
import os
import transformers
import datasets
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

## STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성

In [5]:
nsmc_dataset = load_dataset("nsmc")
nsmc_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

The repository for nsmc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nsmc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [6]:
train_dataset = nsmc_dataset['train']
test_dataset = nsmc_dataset['test']

In [ ]:
train_dataset

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

In [7]:
# train, val로 나누기
dataset_split = train_dataset.train_test_split(test_size = 0.1, seed = 2024)
dataset_split

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 135000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 15000
    })
})

In [8]:
train_dataset = dataset_split['train']
val_dataset = dataset_split['test']

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

135000
15000
50000


## STEP 2. klue/bert-base model 및 tokenizer 불러오기

In [10]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels = 2)

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기


In [11]:
def transform(data):
    return tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        max_length = 128,
        return_token_type_ids = False
    )

# 토큰화 및 패딩 적용
train_dataset = train_dataset.map(transform, batched = True)
val_dataset = val_dataset.map(transform, batched = True)
test_dataset = test_dataset.map(transform, batched = True)

Map:   0%|          | 0/135000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
- 데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.

In [20]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00


In [21]:
from evaluate import load
#from sklearn.metrics import f1_score

accuracy_metric = load("accuracy")
#f1_metric = load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)

    acc = accuracy_metric.compute(references = labels, predictions = predictions)
    #f1 = f1_score(labels, predictions, average = "macro")

    return {
        "accuracy": acc["accuracy"],
        #"f1": f1["f1"]
    }

In [ ]:
# 적정 batch_size / GPU = 16 ~ 32 (단, colab같은 환경에서는 16 이하 적정)
# Epoch: 3 ~ 5 추천

output_dir = "/content/drive/MyDrive/Colab Notebooks"

training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    num_train_epochs = 3,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()

<ipython-input-15-f099f4676fba>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.251700,0.279682,0.900600
2,0.189700,0.269812,0.909400
3,0.123000,0.376479,0.906467


TrainOutput(global_step=25314, training_loss=0.20053614290666455, metrics={'train_runtime': 9104.1365, 'train_samples_per_second': 44.485, 'train_steps_per_second': 2.78, 'total_flos': 2.66399943552e+16, 'train_loss': 0.20053614290666455, 'epoch': 3.0})

In [ ]:
# 3epoch 결과 확인
trainer.evaluate(test_dataset)

{'eval_loss': 0.2755829393863678,
 'eval_accuracy': 0.9068,
 'eval_runtime': 335.4153,
 'eval_samples_per_second': 149.069,
 'eval_steps_per_second': 2.331,
 'epoch': 3.0}

- eval accuracy 값이 0.9068 나왔다! 루브릭 목표 달성!
- 3 epoch에서 과적합이 발생해서 성능이 좋지 않을 것 같았는데 생각보다 나쁘지 않았다.
- max_length 값을 따로 지정하지 않았을 때 보다 약 1.1% 높게 나옴

In [ ]:
del model
del tokenizer

In [ ]:
# 2epoch 결과 확인
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/checkpoint-16876"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, num_labels = 2)

training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    num_train_epochs = 3,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
)

trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-19-3add7f511fa9>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.26981204748153687,
 'eval_model_preparation_time': 0.003,
 'eval_accuracy': 0.9094,
 'eval_runtime': 100.1672,
 'eval_samples_per_second': 149.75,
 'eval_steps_per_second': 2.346}

- 2 epoch의 accuracy 값은 0.9094로 3 epoch의 결과에 비해 약 0.0026 정도 높게 나타남
- 3 epoch까지 학습하는 동안 2epoch에서 최적의 학습이 발생함 (training loss 와 validation loss 감소 함께 발생)

## STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교
- 아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.
    - [Data Collator](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator)
    - [Trainer.TrainingArguments 의 group_by_length](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
# 모델 초기화
del tokenizer
del model

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels = 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 데이터 다시 불러오기...
train_dataset = nsmc_dataset['train']
dataset_split = train_dataset.train_test_split(test_size = 0.1, seed = 2024)

train_dataset = dataset_split['train']
val_dataset = dataset_split['test']
test_dataset = nsmc_dataset['test']

In [ ]:
# 데이터 전처리 코드 수정
def transform(data):
    return tokenizer(
        data["document"],
        truncation = True,
        padding = True, # 동적 패딩 활성화
        max_length = 128, # 최대 길이는 128로 고정
        return_token_type_ids = False
    )

train_dataset = train_dataset.map(transform, batched = True)
val_dataset = val_dataset.map(transform, batched = True)
test_dataset = test_dataset.map(transform, batched = True)

Map:   0%|          | 0/135000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

output_dir = "/content/drive/MyDrive/Colab Notebooks/GD08_250103"

training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    num_train_epochs = 3,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    group_by_length = True # 길이 그룹화 활성화
)

trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-24-ab8dc7d29a83>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.255800,0.250339,0.900600
2,0.186200,0.273536,0.907133
3,0.122900,0.370854,0.907733


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=25314, training_loss=0.19907736810281998, metrics={'train_runtime': 7582.7077, 'train_samples_per_second': 53.411, 'train_steps_per_second': 3.338, 'total_flos': 2.120624128434624e+16, 'train_loss': 0.19907736810281998, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


{'eval_loss': 0.37085404992103577,
 'eval_accuracy': 0.9077333333333333,
 'eval_runtime': 80.9823,
 'eval_samples_per_second': 185.226,
 'eval_steps_per_second': 2.902,
 'epoch': 3.0}

### 동적 패딩 적용 & Max_length 미지정해보기

In [ ]:
del model
del tokenizer

In [ ]:
# 데이터 다시 불러오기...
train_dataset = nsmc_dataset['train']
dataset_split = train_dataset.train_test_split(test_size = 0.1, seed = 2024)

train_dataset = dataset_split['train']
val_dataset = dataset_split['test']
test_dataset = nsmc_dataset['test']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels = 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 데이터 전처리 코드 수정
def transform(data):
    return tokenizer(
        data["document"],
        truncation = True,
        padding = True, # 동적 패딩 활성화
        return_token_type_ids = False
    )

train_dataset = train_dataset.map(transform, batched = True)
val_dataset = val_dataset.map(transform, batched = True)
test_dataset = test_dataset.map(transform, batched = True)

Map:   0%|          | 0/135000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

output_dir = "/content/drive/MyDrive/Colab Notebooks/GD08_250103"

training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    num_train_epochs = 3,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    group_by_length = True # 길이 그룹화 활성화
)

trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-32-ab8dc7d29a83>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.258200,0.249970,0.901600
2,0.180600,0.275828,0.909867
3,0.124200,0.367112,0.908200


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=25314, training_loss=0.19992212598580192, metrics={'train_runtime': 7608.7657, 'train_samples_per_second': 53.228, 'train_steps_per_second': 3.327, 'total_flos': 2.126678971596096e+16, 'train_loss': 0.19992212598580192, 'epoch': 3.0})

In [15]:
del tokenizer
del model

In [16]:
# 데이터 다시 불러오기...
train_dataset = nsmc_dataset['train']
dataset_split = train_dataset.train_test_split(test_size = 0.1, seed = 2024)

train_dataset = dataset_split['train']
val_dataset = dataset_split['test']
test_dataset = nsmc_dataset['test']

In [17]:
# 결과 확인
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/GD08_250103/checkpoint-25314"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, num_labels = 2)

# 데이터 전처리 코드 수정
def transform(data):
    return tokenizer(
        data["document"],
        truncation = True,
        padding = True, # 동적 패딩 활성화
        return_token_type_ids = False
    )

train_dataset = train_dataset.map(transform, batched = True)
val_dataset = val_dataset.map(transform, batched = True)
test_dataset = test_dataset.map(transform, batched = True)


In [22]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/GD08_250103"

training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    num_train_epochs = 3,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
)

trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-22-dbb083e86a2d>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 0.3671119511127472,
 'eval_model_preparation_time': 0.0059,
 'eval_accuracy': 0.9082,
 'eval_runtime': 72.9677,
 'eval_samples_per_second': 205.57,
 'eval_steps_per_second': 3.221}

In [23]:
# 2epoch 결과 가져오기..

# 모델 초기화
del tokenizer
del model

checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/GD08_250103/checkpoint-16876"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, num_labels = 2)

output_dir = "/content/drive/MyDrive/Colab Notebooks/GD08_250103"

training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    num_train_epochs = 3,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
)

trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-23-c22c9edb07b6>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.275828093290329,
 'eval_model_preparation_time': 0.0039,
 'eval_accuracy': 0.9098666666666667,
 'eval_runtime': 76.4017,
 'eval_samples_per_second': 196.331,
 'eval_steps_per_second': 3.076}

## 회고

- Step 4에서 3epoch 훈련했을 때 루브릭 기준 accuracy 0.90을 넘길 수 있었음
- 모델의 성능은 2 > 3 > 1 epoch 순으로 나타남
- 토크나이즈 할 때 동적 패딩을 사용하지 않는다면, max_length를 지정하는 것이 모델 학습에 매우 큰 영향을 미치는 것을 확인함
    - max_length 미지정 -> max_length = 128 지정 시, 학습 속도가 9시간 30분 -> 2시간 30분으로 크게 개선됨
    - 또한, 모델의 성능도 미지정 모델에 비해 accuracy가 1% 정도 향상됨
- Bucketing, 동적 패딩을 적용하고 나서도 학습 속도가 30분이나 단축됨
    - 동적 패딩 적용 & max_length 미지정 모델과 비교했을 때 학습 속도는 크게 차이나지 않음
- PEFT, SFTTrainer를 활용해서 학습 속도를 개선해보면 좋을 듯 함